# Batch run models

In [ ]:
%load_ext lab_black
import os, multiprocessing, time, papermill
from meta import model_cfg, make_batch_cfg, parse_batch_results
from IPython.display import clear_output

## Make configs

In [ ]:
batch_name = "zer_error_test_sgd"
batch_output_dir = "batch_eval/{}/".format(batch_name)

# Gen OG random seed: [int(random.random() * 1e5) for x in range(10)]

param_grid = {"p_noise": [0.0, 1.0, 2.0, 3.0, 4.0], "zero_error_radius": [True, False]}

static_hpar = {
    "rng_seed": 53797,
    "sample_name": "jay",
    "cleanup_units": 20,
    "use_semantic": False,
    "input_dim": 119,
    "output_dim": 250,
    "hidden_units": 100,
    "pretrain_attractor": False,
    "rnn_activation": "sigmoid",
    "w_initializer": 0.1,
    "regularizer_const": None,
    "tau": 1 / 3,
    "max_unit_time": 4.0,
    "output_ticks": 2,
    "optimizer": "sgd",
    "learning_rate": 0.004,
    "n_mil_sample": 5.0,
    "batch_size": 1,
    "save_freq": 10,
    "bq_dataset": batch_name,
}

batch_cfgs = make_batch_cfg(batch_name, static_hpar, param_grid, "OSP_master.ipynb")

## Parallel run

In [ ]:
import nest_asyncio

nest_asyncio.apply()

# Run
def run_batch(cfg):
    """
    Using papermill to run parameterized notebook
    To prevent overwriting, set default overwrite to False if needed
    """

    print("Running model {}".format(cfg["sn"]))
    os.makedirs(cfg["model_folder"], exist_ok=True)
    papermill.execute_notebook(
        cfg["in_notebook"],
        cfg["out_notebook"],
        parameters=cfg["params"],
        nest_asyncio=True,
    )


# Run in parallel pool
with multiprocessing.Pool(4) as pool:
    pool.map(run_batch, batch_cfgs)

clear_output()
print("Done")

Compile and save results

In [ ]:
# Cannot execute together with parallel run for some reason... maybe due to nest_asynio... forgot how to fix...

cfgs, df = parse_batch_results(batch_cfgs)
df.to_csv(batch_output_dir + "bcdf.csv")
cfgs.to_csv(batch_output_dir + "cfgs.csv")

Shutdown compute engine

In [ ]:
time.sleep(30)
print('shuting down')
!sudo poweroff